<a href="https://colab.research.google.com/github/EN10/modded-llama2.c/blob/main/Baby_Llama_128.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%time
# training and inference code
!git clone https://github.com/eniompw/modded-llama2.c

In [ ]:
%%time
!mkdir -p ./modded-llama2.c/data
# Download files in parallel directly into the target directory
!wget -P ./modded-llama2.c/data https://huggingface.co/datasets/enio/TinyStories/resolve/main/tok128/tok128.model & \
  wget -P ./modded-llama2.c/data https://huggingface.co/datasets/enio/TinyStories/raw/main/tok128/tok128.vocab & \
  wget -P ./modded-llama2.c/data https://huggingface.co/datasets/enio/TinyStories/resolve/main/tok128/tok128.tar.gz & \
  wait

# Untar the large file after all downloads are complete
!tar -I pigz -xf ./modded-llama2.c/data/tok128.tar.gz -C ./modded-llama2.c/data/

# compile run / inference executable
!cd modded-llama2.c && gcc -O3 -o run run.c -lm
# create tok105.bin
!cd modded-llama2.c && python tokenizer.py --tokenizer-model=data/tok128.model

In [ ]:
%%time
# train neural net
!cd modded-llama2.c && python train.py --vocab_source=custom --vocab_size=128 --compile=True \
  --dim=128 --n_layers=5 --n_heads=8 --n_kv_heads=4 --batch_size=32 \
  --always_save_checkpoint=True --eval_interval=1 --max_iters=1 #--init_from='resume'

In [ ]:
%%time
# inference
!cd ./modded-llama2.c && ./run out/model.bin -z data/tok128.bin -t 0.8 -n 256 -i "Once upon a time "